In [ ]:
---
title: Measuring the Quality of Answers in Political Q\&As with Large Language Models
author: R. Michael Alvarez and Jacob Morrier
format: dashboard
keep-ipnyb: true
---

from dash import Dash, dcc, html, Input, Output
import math
import pandas as pd
import plotly.express as px

dist_by_topic_CPC = pd.read_csv('Topics_CPC.csv')
dist_by_topic_LPC = pd.read_csv('Topics_LPC.csv')

## Row {height=100%}

In [ ]:
page_max = 20

app = Dash(__name__,
           suppress_callback_exceptions = True)

app.layout = html.Div([
    dcc.Tabs(id="tabs", value='CPC-tab', children=[
        dcc.Tab(label='Conservative Party', value='CPC-tab'),
        dcc.Tab(label='Liberal Party', value='LPC-tab'),
    ]),
    html.Div(id='tabs-content')
])

@app.callback(Output('tabs-content', 'children'),
          Input('tabs', 'value'))
def render_content(tab):
    if tab == 'CPC-tab':
        return html.Div([
          dcc.Graph(id='scatter-plot-CPC'),
          dcc.Slider(
            id='slider-CPC',
            min=1,
            max=math.ceil(len(dist_by_topic_CPC) / page_max),
            step=1,
            value=1)
        ])
    elif tab == 'LPC-tab':
        return html.Div([
          dcc.Graph(id='scatter-plot-LPC'),
          dcc.Slider(
              id='slider-LPC',
              min=1,
              max=math.ceil(len(dist_by_topic_LPC) / page_max),
              step=1,
              value=1),
        ])

@app.callback(
    Output('scatter-plot-CPC', 'figure'),
    Input('slider-CPC', 'value')
)
def update_bar_chart_CPC(value):
    filtered_df = dist_by_topic_CPC.iloc[(page_max * (value - 1)):min(page_max * value, len(dist_by_topic_CPC))]
    filtered_df['sem'] = filtered_df['sem'] * 1.96
    fig = px.scatter(filtered_df, y = 'Llama2', x = 'mean', error_x = 'sem',
                 labels={
                     "Llama2": "Topic",
                     "mean": "Average Cosine Similarity"
                 })
    fig.update_traces(marker=dict(color='blue'))
    return fig

@app.callback(
    Output('scatter-plot-LPC', 'figure'),
    Input('slider-LPC', 'value')
)
def update_bar_chart_LPC(value):
    filtered_df = dist_by_topic_CPC.iloc[(page_max * (value - 1)):min(page_max * value, len(dist_by_topic_LPC))]
    filtered_df['sem'] = filtered_df['sem'] * 1.96
    fig = px.scatter(filtered_df, y = 'Llama2', x = 'mean', error_x = 'sem',
                 labels={
                     "Llama2": "Topic",
                     "mean": "Average Cosine Similarity"
                 })
    fig.update_traces(marker=dict(color='red'))
    return fig

app.run_server(debug=True)

<IPython.core.display.Javascript object>